In [1]:
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
# OPENAI_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv()

True

In [18]:
from serpapi import GoogleSearch

search_keyword = "상일동역 맛집"

params = {
    "engine": "naver",
    "query": search_keyword,
    "api_key": "82af2a26a893c2969e972a0e068a5e847f58972fc422b5a61fb31f5209bc1086",
}

search = GoogleSearch(params)
results = search.get_dict()
results

{'search_metadata': {'id': '68a58ff21dcfe2d9c93cc578',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/33d1b1ed5212a2f8/68a58ff21dcfe2d9c93cc578.json',
  'created_at': '2025-08-20 09:05:54 UTC',
  'processed_at': '2025-08-20 09:05:54 UTC',
  'naver_url': 'https://search.naver.com/search.naver?query=%EC%83%81%EC%9D%BC%EB%8F%99%EC%97%AD+%EB%A7%9B%EC%A7%91',
  'raw_html_file': 'https://serpapi.com/searches/33d1b1ed5212a2f8/68a58ff21dcfe2d9c93cc578.html',
  'total_time_taken': 2.24},
 'search_parameters': {'query': '상일동역 맛집',
  'engine': 'naver',
  'device': 'desktop'},
 'related_results': [{'position': 1,
   'title': '상일동역 맛집 세민수산',
   'link': 'https://search.naver.com/search.naver?where=nexearch&ssc=tab.nx.all&query=%EC%83%81%EC%9D%BC%EB%8F%99%EC%97%AD+%EB%A7%9B%EC%A7%91+%EC%84%B8%EB%AF%BC%EC%88%98%EC%82%B0&sm=tab_she&qdt=0'},
  {'position': 2,
   'title': '상일동역',
   'link': 'https://search.naver.com/search.naver?where=nexearch&ssc=tab.nx.all&query=%EC%83%81%EC%9D

In [17]:
# Gemini 2.5 Flash 모델을 사용하여 네이버 검색 결과에서 식당명을 추출합니다.
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List
import json


# 식당 정보를 담는 모델 정의
class Restaurant(BaseModel):
    name: str = Field(description="식당명")


# 여러 식당 정보를 담는 모델 정의
class RestaurantList(BaseModel):
    restaurants: List[Restaurant]


# OutputParser 초기화
parser = PydanticOutputParser(pydantic_object=RestaurantList)

# Gemini 2.5 Flash 모델 초기화
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# 프롬프트 템플릿 생성
prompt_template = """
다음 네이버 검색 결과에서 식당명만을 정확히 추출해주세요.
각 검색 결과를 분석하여 식당 이름을 정확히 식별하고 JSON 형식으로 출력해주세요.
블로그나 리뷰 사이트가 아닌 실제 식당명만 추출해주세요.

{format_instructions}

네이버 검색 결과:
{search_data}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["search_data"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 체인 구성: 프롬프트 → 모델 → 파서
chain = prompt | model | parser

# 네이버 검색 결과를 문자열로 변환하여 모델에 전달
search_data_str = json.dumps(results, ensure_ascii=False, indent=2)

# Gemini 모델로 식당명 추출
try:
    result_obj = chain.invoke({"search_data": search_data_str})

    # JSON 출력
    json_str = result_obj.model_dump_json(indent=2)
    print("=== JSON 출력 ===")
    print(json_str)

    print("\n=== 식당명 목록 ===")
    # 보기 좋게 출력
    for i, r in enumerate(result_obj.restaurants, 1):
        print(f"{i}. 식당명: {r.name}")
        print("-" * 30)

except Exception as e:
    print(f"오류 발생: {e}")
    print("네이버 검색 결과에서 식당명을 추출할 수 없습니다.")

=== JSON 출력 ===
{
  "restaurants": [
    {
      "name": "세민수산"
    },
    {
      "name": "양미당"
    },
    {
      "name": "고반식당"
    },
    {
      "name": "돈먹소"
    },
    {
      "name": "참치빠"
    },
    {
      "name": "어반나이프"
    },
    {
      "name": "케이리치버거"
    },
    {
      "name": "푼푼 왕돈까스"
    }
  ]
}

=== 식당명 목록 ===
1. 식당명: 세민수산
------------------------------
2. 식당명: 양미당
------------------------------
3. 식당명: 고반식당
------------------------------
4. 식당명: 돈먹소
------------------------------
5. 식당명: 참치빠
------------------------------
6. 식당명: 어반나이프
------------------------------
7. 식당명: 케이리치버거
------------------------------
8. 식당명: 푼푼 왕돈까스
------------------------------
